In [21]:
import requests
from bs4 import BeautifulSoup

base = requests.get("https://www.reddit.com/r/AmItheAsshole/")
if base.status_code != 200:
    raise Exception(base.text)

html = base.text
soup = BeautifulSoup(html, "html.parser")

In [22]:
post_url = 'https://www.reddit.com'+soup.find_all('a', attrs={"slot": "full-post-link"})[2].get("href")
post_id = post_url.split('comments/')[-1].rsplit('/')[0]
post_id

'18t3is2'

In [23]:
post = requests.get(post_url)
if post.status_code != 200:
    raise Exception(post.text)

soup = BeautifulSoup(post.text, "html.parser")

In [47]:
title = soup.find("h1", id="post-title-t3_"+post_id)
user = soup.find("span", slot="authorName")
stamp = soup.find("faceplate-timeago")
title.text #parse text
user.text
stamp.get('ts')

'2023-12-28T20:22:55.465000+0000'

In [61]:
post_div = soup.find("div", id=f"t3_{post_id}-post-rtjson-content")
lines = post_div.text.split('\n')

In [63]:
import os

os.listdir()

['.venv', 'debug.ipynb', 'scrapper', 'test.py']

In [58]:
x = requests.get("https://www.reddit.com/r/AmItheAsshole/comments/18sw9vj/")
test_s = BeautifulSoup(x.text, 'html.parser')
test_s.find("shreddit-redirect").get('href')

'/r/AmItheAsshole/comments/18sw9vj/aita_for_taking_away_my_daughters_gifts_because/'

In [60]:
eval("test_s.find(\"shreddit-redirect\").get(\"href\")")

'/r/AmItheAsshole/comments/18sw9vj/aita_for_taking_away_my_daughters_gifts_because/'

In [1]:
import json

with open("storage/AmItheAsshole/18t3is2.json", "r") as f:
    json_ = json.loads(f.read())

json_

{'author': 'Connect-Bet5983',
 'postId': '18t3is2',
 'postText': 'I’m 30F. My mom is 65F. My sister is 38F.\nMy sister’s son is 10 months old. She’s basically a single mother. She works a full-time job and is the breadwinner, and is also the parent who takes care of the baby the most. The father only works part-time as a waiter to focus on his “art” (making music). He says he can’t look after the kid because he’s tired after his part-time job. They are struggling with money and have downgraded to a smaller apartment to save on rent. My mom has been giving them money every month and buying them baby stuff. Me and my mom (mostly my mom) have been helping her take care of the child. She spends a week out of the month over their apartment to help take care of the baby. My mom comes home and I go over there for the next week. We have been alternating like that. It’s taken a toll on my mom’s mental and physical health. I also don’t like it because I feel like I should be focusing on my own l

In [96]:
import pyttsx3
import re
from time import sleep

subr = json_["readableSubreddit"]
ts = json_["readableStamp"]

engine = pyttsx3.init()
engine.setProperty("rate", 100)
engine.setProperty("pitch", 30)

counter = 1
subfile = """<?xml version="1.0" encoding="UTF-8"?>
<html xmlns="http://www.w3.org/1999/xhtml" xmlns:epub="http://www.idpf.org/2007/ops" lang="en" xml:lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta name="viewport" content="width=768,height=1024"/>
  <link rel="stylesheet" href="../Styles/style.css" type="text/css"/>
  <title>Sonnet I</title>
 </head>
 <body>
  <div id="divTitle">
   <h1><span id="f001">I</span></h1>
  </div>
  <div id="divSonnet"> 
   <p>
"""

engine.setProperty("voice", "mb-us2")
subfile+=f'\t<span id=f{counter:03}">'+(phrase:=f'Post on "{subr}" by "{json_["author"]}". "{ts}"')+'</span><br/>\n'
engine.save_to_file(phrase, f"line_{counter:03}.mp3")
counter+=1
sleep(1)
subfile+=f'\t<span id=f{counter:03}">'+(title:=json_["title"].replace('AITA', subr))+'</span><br/>\n'
engine.save_to_file(title, f"line_{counter:03}.mp3")
counter+=1
sleep(1)
lines = json_["postText"].split('.')
for line in lines:
    to_replace = re.findall(r'(\d+F|\d+M)', line)
    replacement = {}
    for match in to_replace:
        if 'F' in match:
            replacement[match] = match.replace('F', ' Female')
        if 'M' in match:
            replacement[match] = match.replace('M', ' Male')

    for k, v in replacement.items():
        line = line.replace(k, v)
    
    line = line.strip()
    subfile+=f'\t<span id=f{counter:03}">'+line+'</span><br/>\n'
    engine.save_to_file(line, f"line_{counter:03}.mp3")
    counter+=1
engine.runAndWait()
subfile.rsplit("<br/>")[0]
subfile+="""
   </p>
  </div>
 </body>
</html>"""
with open("subs.xhtml", "w") as f:
    f.write(subfile)


In [2]:
import pyttsx3

engine = pyttsx3.init()
engine.say("Hello")
engine.say("how are you?")
engine.save_to_file(filename="test.mp3")

TypeError: Engine.save_to_file() missing 1 required positional argument: 'text'

In [84]:
import re

s = "Me 30F. my mom 65F. and my bro 32M."
to_replace = re.findall(r'(\d+F|\d+M)', s)
replacement = {}
for match in to_replace:
    if 'F' in match:
        replacement[match] = match.replace('F', ' Female')
    if 'M' in match:
        replacement[match] = match.replace('M', ' Male')

for k, v in replacement.items():
    s = s.replace(k, v)

s



'Me 30 Female. my mom 65 Female. and my bro 32 Male.'